# GRNN定位模型

In [11]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score, mean_squared_error

## 读取数据

In [12]:
# 加载训练数据
def load_data(path):
    df = pd.read_csv(path)
    dv = df.values
    X = dv[:, 1:-2]
    Y = dv[:, -2:]
    return X[:, :, np.newaxis], Y

In [13]:
X, y = load_data('./dataset/location_data.csv')
print(X.shape)
X = np.reshape(X, (2000, 100))

(2000, 100, 1)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)  # 训练测试划分

## 定义GRNN模型

In [15]:
class my_GRNN:
    def __init__(self, X_train, y_train, sigma=1.0):
        self.X_train = X_train
        self.y_train = y_train
        self.sigma = sigma

    
    # 1.计算每条样本与所有训练样本的欧式距离
    def distance_mat(self, X_test):
        Eu_dis = []
        for Xi in self.X_train:
            Eu_dis.append(np.linalg.norm(X_test - Xi))
        return Eu_dis

    
    # 2.模式层
    def Gauss(self, Eu_dis):
        g_list = []
        for ei in Eu_dis:
            gi = math.exp(-ei / (2 * (self.sigma ** 2)))
            g_list.append(gi)
        return g_list

    
    # 3.求和层
    def sum_layer(self, g_list):
        s_list = []
        s0 = sum(g_list)
        s_list.append(s0)
        g_list = np.array(g_list)
        m, n = np.shape(self.y_train)
        for i in range(n):
            s_list.append(np.sum(g_list * self.y_train[:, i]))
        return s_list

    
    # 4.输出层
    def output_layer(self, s_list):
        ans = []
        for i in range(1, len(s_list)):
            ans.append(s_list[i] / s_list[0])
        return ans

    
    # 模型整体架构
    def process_GRNN(self, X_test):
        pxy = []
        for xt in X_test:
            Eu_dis = self.distance_mat(xt)
            g_list = self.Gauss(Eu_dis)
            s_list = self.sum_layer(g_list)
            pxy.append(self.output_layer(s_list))
        return pxy
    

## 测试执行模型

In [16]:
%%time
print(y_test[:10])
GRNN = my_GRNN(X_train, y_train, sigma=0.5)
pxy = GRNN.process_GRNN(X_test)
print(pxy[:10])
MAE = mean_absolute_error(y_test, pxy)
MSE = mean_squared_error(y_test, pxy)
r2 = r2_score(y_test, pxy)
ev = explained_variance_score(y_test, pxy)
print('MAE指标:', MAE)
print('MSE指标:', MSE)
print('RMSE指标:', math.sqrt(MSE))
print('r2指标:', r2)
print('ev指标:', ev)

[[3.76 3.77]
 [1.57 4.63]
 [5.43 6.77]
 [9.67 7.94]
 [7.09 8.28]
 [8.59 8.24]
 [9.14 2.22]
 [3.66 7.91]
 [8.93 1.86]
 [2.38 2.02]]
[[4.313237194213279, 3.3405733852711235], [2.0623590256770457, 4.8355333079211125], [5.016479948035213, 8.15589142044068], [8.774914492673199, 7.60577423172001], [6.343263714332327, 8.819755709700623], [9.120573285434979, 7.733082746961303], [8.951306263082238, 5.016816758957814], [4.031207390570993, 8.693088076344546], [9.216440713710975, 3.048697034087624], [2.396122470294987, 2.702815885434046]]
MAE指标: 0.43088606583294475
MSE指标: 0.3623140942733079
RMSE指标: 0.601925322837732
r2指标: 0.9498994247458706
ev指标: 0.9504105349716667
CPU times: user 3.33 s, sys: 49.5 ms, total: 3.38 s
Wall time: 3.36 s
